<a href="https://colab.research.google.com/github/jlplautz/IntensivaoPython/blob/master/Aula1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_excel(r'/content/drive/MyDrive/Projetos Intensivao Python/Vendas.xlsx')
display(df)

Passo 1 - Faturamento das Lojas

In [ ]:
faturamento = df[['ID Loja', 'Valor Final']].groupby("ID Loja").sum()
faturamento = faturamento.sort_values(by='Valor Final', ascending=False)
display(faturamento)

Passo 2 - Calculando a qtdade vendida por loja


In [11]:
quantidade = df[['ID Loja', 'Quantidade']].groupby('ID Loja').sum()
quantidade - quantidade.sort_values(by='ID Loja', ascending=False)
display(quantidade)

,Quantidade
ID Loja,
Bourbon Shopping SP,8863
Center Shopping Uberlândia,8627
Iguatemi Campinas,8935
Iguatemi Esplanada,8580
Norte Shopping,9014
Novo Shopping Ribeirão Preto,8751
Palladium Shopping Curitiba,9091
Parque Dom Pedro Shopping,8388
Passei das Águas Shopping,8618


passo 3 - Calculando o ticket Medio dos Produtos por Loja

In [ ]:
ticket_medio = (faturamento['Valor Final'] / quantidade['Quantidade']).to_frame()
ticket_medio = ticket_medio.rename(columns={0: 'Ticket Medio'})
display(ticket_medio)

Passo 4 - Criando a função de enviar email

Vamos revisar o que fizemos até agora e que informações temos
disponíveis.
1. Obtemos os dados da base Vendas.xlsx;
2. Organizamos os dados usando o PANDAS;
3. Extraímos da base de dados original df, 3 tabelas específicas:
   *   Faturamento por loja;
   *   Quantidade vendida por loja;
   *  Ticket médio por loja.

4. Ordenamos todos os dados, do maior para o menor, para facilitar 
  Agora que temos os dados, vamos para parte do relatório. Nosso
  objetivo é que cada um dos gerentes de loja receba, no e-mail, um
  relatório com as informações mais importantes da sua loja.

Para isso, vamos precisar criar uma função que:
1. Colete as informações que geramos;
2. Crie o e-mail com os dados da loja;
3. Envie o e-mail para o gerente daquela loja específica;
4. Enviar e-mail para Diretoria.


In [18]:
# função enviar_email
import smtplib
import email.message

def enviar_email(resumo_loja, loja):

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f"""
  <p> Jorge Plautz,</p>
  {resumo_loja.to_html()}
  <p>Qualquer duvida estamos a disposição</p>
  """
  
  msg = email.message.Message()
  msg['Subject'] = f'Jorge Plautz - Loja: {loja}'

  msg['From'] = 'jorge.plautz@gmail.com'
  msg['To'] = 'jorge.plautz@gmail.com'
  password = "Jlg2r7.1"
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email)
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')

Passo 5 - Criando relátorio por loja

Primeiramente, vamos criar nossa variável loja que será necessária na nossa função enviar_email.

Para isso, vamos usar a coluna ‘ID loja’ do nosso dataframe df.

Como podemos ver, essa coluna possui valores repetidos, portanto, vamos usar um método que nos permite ter uma lista de
valores únicos. Esse método é o^:
.unique().

Como temos 25 lojas distintas, escrever um código específico para cada uma delas não parece ser a forma mais prática e eficiente de
se agir.

Imagine que ao invés de 25 fossem 500? Não parece prático né?

Para isso, temos estruturas de repetição no Python. A mais comum delas é o FOR. Ela nos permite percorrer todas as nossas 25 lojas e
criar um relatório de forma automática.

Aqui usaremos a indentação novamente. Logo, todas as linhas que estão indentadas pertencem ao bloco for.

A estrutura loja in lojas, significa que uma loja por vez da lista lojas irá passar pelas 3 linhas pertencentes ao for.

Abaixo do código, fazemos duas iterações teóricas para que fique mais claro


Enviando o e-mail por loja

Podemos aproveitar as iterações para já enviarmos o e-mail da loja .
Para isso, vamos usar nossa função enviar_email!

Vamos lembrar quais são seus argumentos:
Enviar_email(resumo_loja , loja)
Podemos perceber que resumo_loja é calculado dentro do FOR.

Já loja, como vimos no slide anterior, irá variar até que o Python passe por todas as lojas.


In [ ]:
lojas = df['ID Loja'].unique()

for loja in lojas:
  tabela_loja = df.loc[df['ID Loja'] == loja, ['ID Loja', 'Quantidade', 'Valor Final']]
  resumo_loja = tabela_loja.groupby('ID Loja').sum()
  resumo_loja['Ticket_Medio'] = resumo_loja['Valor Final'] / resumo_loja['Quantidade']
  enviar_email(resumo_loja, loja)

Passo 6 - Enviando email para a direoria

Para a diretoria iremos seguir o mesmo conceito, mas por se tratar de um nível hierárquico maior, vamos agrupar toda a informação em uma única tabela para que seja possível uma visão macro de como vão
os resultados da empresa.

Para isso, usaremos nossas variáveis iniciais:
*   faturamento;
*   quantidade;
*   ticket_médio;
 
Para unirmos as três informações usaremos o método .join() conforme apresentado ao lado.

In [19]:
# email para diretoria

tabela_diretoria = faturamento.join(quantidade).join(ticket_medio)
enviar_email(tabela_diretoria, 'Todas as Lojas')


Email enviado
